### import lib

In [ ]:
from os import sep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
import pandas as pd


### setup and open chrome
### put chrome.exe driver to Programfile forder or link absolute path on Options()

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

### get point a student

In [83]:
# get input field and submit
def getSV(msv=637871) :
    msv_field = driver.find_element_by_id('ctl00_ContentPlaceHolder1_ctl00_txtMaSV')
    msv_field.send_keys(msv)
    search_btn = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_btnOK"]')
    
    search_btn.click()
    sleep(2)

    # view all result
    view_all = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_lnkChangeview2"]')
    view_all.click()

# get list point each stu
def getResult() :
    all_point = {}
    page_source = BeautifulSoup(driver.page_source)
    list_row = page_source.find_all('tr', class_='row-diem')
    for j, row in enumerate(list_row) :
        if j == len(list_row)/2 : break
        for i, cell in enumerate(row) :
            if i==2 : 
                subj = cell.text
            if i==11 : 
                point = cell.text
        while subj in all_point :
            subj = subj + '*'
        all_point[subj] = point  
    sv = driver.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_ctl00_ucThongTinSV_lblMaSinhVien"]').text
    all_point['MSV'] = sv
    return all_point


In [89]:
inputFile = pd.read_csv('data.csv', header=None,index_col=False).transpose()
final_Result = []
for msv in inputFile.iloc[0]:
    url = "http://daotao.vnua.edu.vn/default.aspx?page=nhapmasv&flag=XemDiemThi"
    driver.get(url)
    sleep(2)
    getSV(msv)
    sleep(2)
    final_Result.append(getResult())

# lean nan value and move col MSV to first
outputFile = pd.DataFrame(final_Result)
outputFile = outputFile.fillna('')
first_column = outputFile.pop('MSV')
outputFile.insert(0, 'MSV', first_column)

outputFile.to_csv('output.csv', index=False)


In [87]:

outputFile

,MSV,SN00010,TH01001,TH01006,QS01002,TH01024,TH01029,GT01016,ML01009,QS01001,...,GT01019,KN01004,TH03201,SN00011*,KN01005,MT02011,KT03024,GT01020,KN01006,TH03207
0,637871,,5.7,4.0,5.6,6.6,4.0,5.5,5.8,5.9,...,,,,,,,,,,
1,637822,,5.1,7.9,6.6,5.5,6.1,5.3,6.5,6.1,...,4.0,7.7,7.9,5.7,8.8,,,,,
2,637634,,5.3,5.5,6.1,8.5,7.6,4.2,6.1,6.7,...,,,6.5,,,,,5.5,8.5,
